In [1]:
# Source: https://github.com/keras-rl/keras-rl

# Normalized Advantage Function (NAF) agents is a way of extending DQN to a continuous action space.
# In NAF the Q-function is decomposed into an advantage term A and state value term V

# Requirements:
# pip install tensorflow
# pip install gym
# pip install keras-rl2

import numpy as np
import gym

from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Flatten, Input, Concatenate
from keras.optimizers import Adam

from rl.agents import NAFAgent
from rl.memory import SequentialMemory
from rl.random import OrnsteinUhlenbeckProcess
from rl.core import Processor

In [2]:
# Initializing PARAMETERS
num_frames = 1
theta = .15
mu = 0.
sigma = .3
learning_rate = 0.001
gamma = .99
target_model_update = 1e-3
memory_limit = 100000
time_steps = 10000
episodes = 2

In [3]:
# Get the environment and extract the number of actions.
env = gym.make('CarRacing-v0')

# getting the number of actions in the action space
num_actions = env.action_space.shape[0]

# extract action inputs and observation inputs
action_input = Input(shape=(num_actions,), name='action_input')
observation_input = Input(shape=(num_frames,) + env.observation_space.shape, name='observation_input')

print(action_input.shape)
print(observation_input.shape)

(None, 3)
(None, 1, 96, 96, 3)


/home/blwme/anaconda3/lib/python3.8/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [4]:
# Building a V-model to learn the state value terms
def build_V_model(environment, summary=False):
    V_model = Sequential()
    V_model.add(Flatten(input_shape=(num_frames,) + environment.observation_space.shape))
    V_model.add(Dense(16))
    V_model.add(Activation('relu'))
    V_model.add(Dense(16))
    V_model.add(Activation('relu'))
    V_model.add(Dense(16))
    V_model.add(Activation('relu'))
    V_model.add(Dense(1))
    V_model.add(Activation('linear'))
    if summary:
        print(V_model.summary())
    return V_model


# Bulding the mu-model. The mu-model is always the action that maximizes the Q function
# Since the mu-model chooses the action deterministically,
# but adding random_process balances exploration and exploitation.
def build_mu_model(environment, summary=False):
    mu_model = Sequential()
    mu_model.add(Flatten(input_shape=(num_frames,) + env.observation_space.shape))
    mu_model.add(Dense(16))
    mu_model.add(Activation('relu'))
    mu_model.add(Dense(16))
    mu_model.add(Activation('relu'))
    mu_model.add(Dense(16))
    mu_model.add(Activation('relu'))
    mu_model.add(Dense(num_actions))
    mu_model.add(Activation('linear'))
    if summary:
        print(mu_model.summary())
    return mu_model


# Bulding a L-model to learn advantage term A
def build_L_model(action_input, observation_input, summary=False):
    x = Concatenate()([action_input, Flatten()(observation_input)])
    x = Dense(32)(x)
    x = Activation('relu')(x)
    x = Dense(32)(x)
    x = Activation('relu')(x)
    x = Dense(32)(x)
    x = Activation('relu')(x)
    x = Dense(((num_actions * num_actions + num_actions) // 2))(x)
    x = Activation('linear')(x)
    L_model = Model(inputs=[action_input, observation_input], outputs=x)
    if summary:
        print(L_model.summary())
    return L_model



In [5]:
V_model = build_V_model(env, summary=True)
mu_model = build_mu_model(env, summary=True)
L_model = build_L_model(action_input, observation_input, summary=True)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 27648)             0         
_________________________________________________________________
dense (Dense)                (None, 16)                442384    
_________________________________________________________________
activation (Activation)      (None, 16)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                272       
_________________________________________________________________
activation_1 (Activation)    (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 16)                272       
_________________________________________________________________
activation_2 (Activation)    (None, 16)                0

In [6]:
# Configuring and compiling the Agent
memory = SequentialMemory(limit=memory_limit, window_length=num_frames)
random_process = OrnsteinUhlenbeckProcess(theta=theta, mu=mu, sigma=sigma, size=num_actions)
agent = NAFAgent(nb_actions=num_actions, V_model=V_model, L_model=L_model, mu_model=mu_model,
                 memory=memory, nb_steps_warmup=100, random_process=random_process,
                 gamma=gamma, target_model_update=target_model_update)

agent.compile(Adam(lr=learning_rate, clipnorm=1.), metrics=['mae'])

In [13]:
# Training the model (with visualizatoin but turn it off to speed up the learning process)
#agent.fit(env, nb_steps=time_steps, visualize=True, verbose=1, nb_max_episode_steps=episodes)

Training for 10000 steps ...
Track generation: 958..1207 -> 249-tiles track
Interval 1 (0 steps performed)
   29/10000 [..............................] - ETA: 14:06 - reward: 3.3949Track generation: 1247..1566 -> 319-tiles track
retry to generate track (normal if there are not manyinstances of this message)
Track generation: 1207..1513 -> 306-tiles track
   36/10000 [..............................] - ETA: 14:05 - reward: 3.2458Track generation: 1079..1356 -> 277-tiles track
retry to generate track (normal if there are not manyinstances of this message)
Track generation: 1056..1332 -> 276-tiles track
   46/10000 [..............................] - ETA: 14:59 - reward: 3.3006Track generation: 1183..1485 -> 302-tiles track
retry to generate track (normal if there are not manyinstances of this message)
Track generation: 1261..1581 -> 320-tiles track
   83/10000 [..............................] - ETA: 14:38 - reward: 3.3537Track generation: 864..1088 -> 224-tiles track
retry to generate trac

  124/10000 [..............................] - ETA: 24:16 - reward: 91.4223Track generation: 1152..1463 -> 311-tiles track
retry to generate track (normal if there are not manyinstances of this message)
Track generation: 1088..1373 -> 285-tiles track
  144/10000 [..............................] - ETA: 25:18 - reward: 148.1553Track generation: 1098..1362 -> 264-tiles track
retry to generate track (normal if there are not manyinstances of this message)
Track generation: 985..1239 -> 254-tiles track
retry to generate track (normal if there are not manyinstances of this message)
Track generation: 1196..1499 -> 303-tiles track
  150/10000 [..............................] - ETA: 25:24 - reward: 162.2251Track generation: 978..1228 -> 250-tiles track
retry to generate track (normal if there are not manyinstances of this message)
Track generation: 1143..1438 -> 295-tiles track
  162/10000 [..............................] - ETA: 26:01 - reward: 187.2381Track generation: 1181..1484 -> 303-tiles t

  258/10000 [..............................] - ETA: 28:07 - reward: 303.5774Track generation: 1030..1293 -> 263-tiles track
retry to generate track (normal if there are not manyinstances of this message)
Track generation: 1246..1526 -> 280-tiles track
retry to generate track (normal if there are not manyinstances of this message)
Track generation: 1276..1599 -> 323-tiles track
  262/10000 [..............................] - ETA: 28:08 - reward: 306.5747Track generation: 1066..1318 -> 252-tiles track
retry to generate track (normal if there are not manyinstances of this message)
Track generation: 1252..1569 -> 317-tiles track
  286/10000 [..............................] - ETA: 28:20 - reward: 322.7978Track generation: 1186..1490 -> 304-tiles track
retry to generate track (normal if there are not manyinstances of this message)
Track generation: 1129..1422 -> 293-tiles track
  310/10000 [..............................] - ETA: 28:22 - reward: 336.5089Track generation: 1185..1489 -> 304-tile

  378/10000 [>.............................] - ETA: 28:30 - reward: 365.9020Track generation: 1008..1267 -> 259-tiles track
retry to generate track (normal if there are not manyinstances of this message)
Track generation: 1140..1429 -> 289-tiles track
  438/10000 [>.............................] - ETA: 28:35 - reward: 384.2579Track generation: 1125..1413 -> 288-tiles track
retry to generate track (normal if there are not manyinstances of this message)
Track generation: 1053..1320 -> 267-tiles track
  454/10000 [>.............................] - ETA: 28:55 - reward: 388.3334Track generation: 1045..1312 -> 267-tiles track
retry to generate track (normal if there are not manyinstances of this message)
Track generation: 1069..1340 -> 271-tiles track
  466/10000 [>.............................] - ETA: 28:52 - reward: 391.2064Track generation: 1331..1669 -> 338-tiles track
retry to generate track (normal if there are not manyinstances of this message)
Track generation: 1116..1405 -> 289-tile

  582/10000 [>.............................] - ETA: 28:49 - reward: 412.8704Track generation: 988..1242 -> 254-tiles track
retry to generate track (normal if there are not manyinstances of this message)
Track generation: 1238..1551 -> 313-tiles track
  606/10000 [>.............................] - ETA: 28:45 - reward: 416.3171Track generation: 940..1183 -> 243-tiles track
retry to generate track (normal if there are not manyinstances of this message)
Track generation: 1060..1329 -> 269-tiles track


...

 9138/10000 [==========================>...] - ETA: 2:52 - reward: 494.3571Track generation: 1161..1463 -> 302-tiles track


 9166/10000 [==========================>...] - ETA: 2:47 - reward: 494.3740Track generation: 1004..1253 -> 249-tiles track
retry to generate track (normal if there are not manyinstances of this message)
Track generation: 1112..1402 -> 290-tiles track
 9168/10000 [==========================>...] - ETA: 2:46 - reward: 494.3752Track generation: 1090..1400 -> 310-tiles track
retry to generate track (normal if there are not manyinstances of this message)
Track generation: 1116..1406 -> 290-tiles track
 9190/10000 [==========================>...] - ETA: 2:42 - reward: 494.3884Track generation: 1074..1349 -> 275-tiles track
retry to generate track (normal if there are not manyinstances of this message)
Track generation: 1092..1369 -> 277-tiles track
 9206/10000 [==========================>...] - ETA: 2:39 - reward: 494.3980Track generation: 908..1144 -> 236-tiles track
retry to generate track (normal if there are not manyinstances of this message)
Track generation: 996..1249 -> 253-tiles trac

 9414/10000 [===========================>..] - ETA: 1:57 - reward: 494.5196Track generation: 1134..1423 -> 289-tiles track
retry to generate track (normal if there are not manyinstances of this message)
Track generation: 1143..1432 -> 289-tiles track
 9432/10000 [===========================>..] - ETA: 1:53 - reward: 494.5299Track generation: 1085..1363 -> 278-tiles track
retry to generate track (normal if there are not manyinstances of this message)
Track generation: 1020..1279 -> 259-tiles track
 9506/10000 [===========================>..] - ETA: 1:39 - reward: 494.5717Track generation: 1094..1372 -> 278-tiles track
retry to generate track (normal if there are not manyinstances of this message)
Track generation: 1232..1544 -> 312-tiles track
 9590/10000 [===========================>..] - ETA: 1:22 - reward: 494.6183Track generation: 1057..1324 -> 267-tiles track
retry to generate track (normal if there are not manyinstances of this message)
Track generation: 1114..1397 -> 283-tiles tr

 9648/10000 [===========================>..] - ETA: 1:10 - reward: 494.6501Track generation: 1063..1335 -> 272-tiles track
retry to generate track (normal if there are not manyinstances of this message)
Track generation: 1121..1409 -> 288-tiles track
retry to generate track (normal if there are not manyinstances of this message)
Track generation: 1083..1365 -> 282-tiles track
 9720/10000 [============================>.] - ETA: 56s - reward: 494.6890Track generation: 1032..1297 -> 265-tiles track
retry to generate track (normal if there are not manyinstances of this message)
Track generation: 1143..1433 -> 290-tiles track
 9762/10000 [============================>.] - ETA: 47s - reward: 494.7114Track generation: 1165..1460 -> 295-tiles track


 9776/10000 [============================>.] - ETA: 45s - reward: 494.7188Track generation: 1106..1390 -> 284-tiles track
retry to generate track (normal if there are not manyinstances of this message)
Track generation: 1118..1410 -> 292-tiles track
 9810/10000 [============================>.] - ETA: 38s - reward: 494.7368Track generation: 1085..1364 -> 279-tiles track
retry to generate track (normal if there are not manyinstances of this message)
Track generation: 1215..1523 -> 308-tiles track
 9958/10000 [============================>.] - ETA: 8s - reward: 494.8135Track generation: 1014..1275 -> 261-tiles track
retry to generate track (normal if there are not manyinstances of this message)
Track generation: 1132..1423 -> 291-tiles track
 9960/10000 [============================>.] - ETA: 8s - reward: 494.8145Track generation: 1218..1530 -> 312-tiles track
retry to generate track (normal if there are not manyinstances of this message)
Track generation: 1273..1595 -> 322-tiles track
 9

In [14]:
# saving the weights
#agent.save_weights('cdqn_{}_weights.h5f'.format('CarRacing-v0'), overwrite=True)

In [7]:
# loading the weights
agent.load_weights('cdqn_CarRacing-v0_weights.h5f')

In [8]:
# Finally, evaluate our algorithm for 10 episodes.
agent.test(env, nb_episodes=10, visualize=True, nb_max_episode_steps=200)

Testing for 10 episodes ...
Track generation: 1055..1323 -> 268-tiles track


/home/blwme/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


Episode 1: reward: 999.800, steps: 2
Track generation: 1047..1313 -> 266-tiles track
Episode 2: reward: 999.800, steps: 2
Track generation: 1103..1383 -> 280-tiles track
Episode 3: reward: 999.800, steps: 2
Track generation: 1141..1430 -> 289-tiles track
Episode 4: reward: 999.800, steps: 2
Track generation: 1088..1364 -> 276-tiles track
Episode 5: reward: 999.800, steps: 2
Track generation: 1284..1609 -> 325-tiles track
Episode 6: reward: 999.800, steps: 2
Track generation: 1115..1398 -> 283-tiles track
Episode 7: reward: 999.800, steps: 2
Track generation: 1234..1556 -> 322-tiles track
Episode 8: reward: 999.800, steps: 2
Track generation: 1197..1509 -> 312-tiles track
Episode 9: reward: 999.800, steps: 2
Track generation: 1250..1567 -> 317-tiles track
Episode 10: reward: 999.800, steps: 2
